<a href="https://colab.research.google.com/github/w-okada/voice-changer/blob/dev/VoiceChangerDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Voice Changer (デモ版)
---

このノートはVoice ChangerをColab上で動かすデモ版です。

正式版はローカルPCのDocker上で動かすアプリケーションです。

正式版は、多くの場合より少ないタイムラグで滑らかに音声を変換できます。

詳細な使用方法はこちらの[リポジトリ](https://github.com/w-okada/voice-changer)からご確認ください。


# GPUを確認
GPUを用いたほうが高速に処理が行えます。

下記のコマンドでGPUが確認できない場合は、上のメニューから

「ランタイム」→「ランタイムの変更」→「ハードウェア アクセラレータ」

でGPUを選択してください。

In [1]:
# (1) GPUの確認
!nvidia-smi

Fri Sep  2 03:08:06 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 使用するモデルとコンフィグファイルの指定

使用するトレーニング済みのモデルと、トレーニングで使用したコンフィグファイルのパスを指定してください。

多くの場合はGoogle Driveに格納されているファイルを使用すると思います。その場合は、下の(2-2)のセルを実行してドライブをマウントしてください

In [2]:
# (2-1) 使用するモデルとコンフィグファイルの指定
CONFIG="/content/drive/MyDrive/VoiceChanger/config.json"
MODEL="/content/drive/MyDrive/VoiceChanger/G_326000.pth"

In [3]:
# (2-2) Google Driveのマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# リポジトリのクローン
リポジトリをクローンします

In [4]:
# (3) リポジトリのクローン
!git clone https://github.com/w-okada/voice-changer.git
%cd voice-changer/demo/


Cloning into 'voice-changer'...
remote: Enumerating objects: 354, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 354 (delta 29), reused 40 (delta 17), pack-reused 278
Receiving objects: 100% (354/354), 20.87 MiB | 29.89 MiB/s, done.
Resolving deltas: 100% (165/165), done.
/content/voice-changer/demo


# ファイルの配置
このデモでは、GUIをWebサーバで提供します。WebサーバからGUIのファイルにアクセスできるようにファイルをコピーします(4-1)。

また、アプリケーションの挙動を記した設定ファイルもコピーします(4-2)。(4-3)はコピーした設定ファイルを表示しています。もしかしたらうまく動かないときに役立つかもしれません。

In [5]:
# (4-1) GUIファイルの配置 
!mkdir -p ../frontend/dist
!cp -r ../docs/* ../frontend/dist/
!ls ../frontend/dist


assets	coffee.png  favicon.ico  index.html  index.js  index.js.LICENSE.txt


In [6]:
# (4-2) 設定ファイルの配置
!cp ../template/setting_colab.json ../frontend/dist/assets/setting.json


In [7]:
# (4-3) 設定ファイルの確認
!cat ../frontend/dist/assets/setting.json


{
    "app_title": "voice-changer",
    "majar_mode": "colab",
    "voice_changer_server_url": "./test",
    "sample_rate": 48000,
    "buffer_size": 1024,
    "prefix_chunk_size": 24,
    "chunk_size": 24,
    "speaker_ids": [100, 107, 101, 102, 103],
    "speaker_names": ["ずんだもん", "user", "そら", "めたん", "つむぎ"],
    "src_id": 107,
    "dst_id": 100,
    "vf_enable": true,
    "voice_changer_mode": "realtime",
    "gpu": 0,
    "available_gpus": [-1, 0, 1, 2, 3, 4],
    "avatar": {
        "enable_avatar": true,        
        "motion_capture_face": true,
        "motion_capture_upperbody": true,
        "lip_overwrite_with_voice": true,
        "avatar_url": "./assets/vrm/zundamon/zundamon.vrm",
        "backgournd_image_url": "./assets/images/bg_natural_sougen.jpg",
        "background_color": "#0000dd",
        "chroma_key": "#0000dd",
        "avatar_canvas_size": [1280, 720],
        "screen_canvas_size": [1280, 720]
    },
    "advance": {
        "avatar_draw_skip_rate": 3,
     

# モジュールのインストール

必要なモジュールをインストールします。

In [8]:
# (5) 設定ファイルの確認
!apt-get install -y espeak libsndfile1-dev
!pip install flask
!pip install python-socketio
!pip install eventlet
!pip install unidecode
!pip install phonemizer
!pip install retry
!pip install flask
!pip install flask_cors


Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsndfile1-dev is already the newest version (1.0.28-4ubuntu0.18.04.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  espeak-data libespeak1 libportaudio2 libsonic0
The following NEW packages will be installed:
  espeak espeak-data libespeak1 libportaudio2 libsonic0
0 upgraded, 5 newly installed, 0 to remove and 20 not upgraded.
Need to get 1,219 kB of archives.
After this operation, 3,031 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libsonic0 amd64 0.2.0-6 [13.4 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 espeak-data amd64 1.48.04+dfsg-5 [934 kB]
Get:4 http://archive.ubuntu.co

# サーバの起動

サーバを起動します。(6-1)

サーバの起動状況を確認します。(6-2) 

このセルは繰り返し実行することになるのでCtrl+Retでセルを実行してください。

アクセスできるようになるまで、１～２分かかるようです。コーヒーでも飲みに行きましょう。

下記のようなテキストが表示されたら起動完了です。

```
[2022-08-31 06:48:38,064] INFO in serverFlask: INITIALIZE MODEL
[2022-08-31 06:48:45,660] INFO in utils: Loaded checkpoint '/content/drive/MyDrive/VoiceChanger/G_326000.pth' (iteration 1136)
[2022-08-31 06:48:57,329] INFO in serverFlask: START APP
GPU_NUM: 1
 * Serving Flask app "serverFlask" (lazy loading)
 * Environment: production
   WARNING: This is a development server. Do not use it in a production deployment.
   Use a production WSGI server instead.
 * Debug mode: on
[2022-08-31 06:48:57,336] INFO in _internal:  * Running on http://0.0.0.0:8082/ (Press CTRL+C to quit)
[2022-08-31 06:48:57,337] INFO in _internal:  * Restarting with stat
[2022-08-31 06:48:59,283] INFO in serverFlask: INITIALIZE MODEL
[2022-08-31 06:49:01,125] INFO in utils: Loaded checkpoint '/content/drive/MyDrive/VoiceChanger/G_326000.pth' (iteration 1136)
[2022-08-31 06:49:04,996] INFO in serverFlask: START APP
[2022-08-31 06:49:05,002] WARNING in _internal:  * Debugger is active!
[2022-08-31 06:49:05,074] INFO in _internal:  * Debugger PIN: 879-442-767
```



In [9]:
# (6-1) サーバの起動
get_ipython().system_raw(f'python3 serverFlask.py 8082 {CONFIG} {MODEL} >foo 2>&1 &')

In [18]:
# (6-2) サーバの起動確認 (Ctrl+Retで実行)
!cat foo

[2022-09-02 03:09:26,807] INFO in serverFlask: INITIALIZE MODEL
[2022-09-02 03:09:32,030] INFO in utils: Loaded checkpoint '/content/drive/MyDrive/VoiceChanger/G_326000.pth' (iteration 1136)
[2022-09-02 03:09:44,106] INFO in serverFlask: START APP
GPU_NUM: 1
 * Serving Flask app "serverFlask" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
[2022-09-02 03:09:44,114] INFO in _internal:  * Running on http://0.0.0.0:8082/ (Press CTRL+C to quit)
[2022-09-02 03:09:44,115] INFO in _internal:  * Restarting with stat
[2022-09-02 03:09:45,942] INFO in serverFlask: INITIALIZE MODEL
[2022-09-02 03:09:47,689] INFO in utils: Loaded checkpoint '/content/drive/MyDrive/VoiceChanger/G_326000.pth' (iteration 1136)
[2022-09-02 03:09:51,469] INFO in serverFlask: START APP
[2022-09-02 03:09:51,474] WARNING in _internal:  * Debugger is active!
[2022-09-02 03:09:51,544] INFO in _internal:  * Debugger PIN: 334-166-753


# プロキシを起動
ウェブサーバへのアクセスをするためのプロキシを起動します。

表示されたURLをクリックして開くと別タブでアプリが開きます。

Colabなので、ロードにある程度時間がかかります(30秒くらい)。

In [19]:
# (7) プロキシを起動
from google.colab import output

output.serve_kernel_port_as_window(8082)

<IPython.core.display.Javascript object>